# 2d image manifold results

## Setup

In [1]:
import sys
import numpy as np
from itertools import product


## Load results

In [2]:
n_runs = 10

In [3]:
algo_filenames = []
algo_additionals = []
algo_labels = []
algo_dividers = []

def add_algo(filename, add, label):
    algo_filenames.append(filename)
    algo_additionals.append(add)
    algo_labels.append(label)
    
    
def add_divider():
    algo_dividers.append(len(algo_filenames))

add_algo("flow", "_april", r"\af{}")
add_algo("pie", "_april", r"\pie{}")
add_algo("mf", "_april", r"\mfmfmd{}")
add_algo("emf", "_april", r"\mfmfemd{}")

n_algos = len(algo_filenames)


In [4]:
def load(name, shape, numpyfy=True, chains=1, result_dir="../data/results"):
    all_results = []
    
    for algo_filename, algo_add, algo_dim in zip(algo_filenames, algo_additionals, algo_dims):
        algo_results = []
            
        for run in range(n_runs):
            run_str = "" if run == 0 else "_run{}".format(run)

            try:
                this_result = np.load(
                    "{}/{}_{}_gan2d_{}.npy".format(result_dir, algo_filename, algo_add, run_str)
                )
                if (not numpyfy) or (shape is None) or np.product(this_result.shape) == np.product(shape):
                    algo_results.append(this_result.reshape(shape))
                else:
                    algo_results.append(np.nan*np.ones(shape))

            except FileNotFoundError as e:
                # print(e)
                if shape is None:
                    algo_results.append(None)
                else:
                    algo_results.append(np.nan*np.ones(shape))
            
        all_results.append(algo_results)
    
    if numpyfy:
        all_results = np.array(all_results, dtype=np.float)
        all_results = all_results.reshape([all_results.shape[0], n_runs, n_trueparams] + list(shape))
        
    return all_results


model_test_reco_xs = load("x_reco", (100, 3, 64, 64))
model_test_reco_errors = load("reco_error", (1000,))
model_gen_fids = load("fid", (1,))


In [ ]:
max_reco_error = 1000.
model_mean_reco_errors = np.mean(np.clip(model_test_reco_error, 0., max_reco_error), axis=2)
model_mean_reco_errors.shape


## Outlier removal

In [37]:
def mean_err_without_outliers(data, remove=1):
    shape = list(data.shape)[:-1]
    data.reshape((-1, data.shape[-1]))
    
    means, errors = [], []
    
    for data_ in data:
        data_ = data_[np.isfinite(data_)]
        if not len(data_) > 0:
            means.append(np.nan)
            errors.append(np.nan)
            continue
            
        if len(data_) > 2*remove + 1:
            for _ in range(remove):
                data_ = np.delete(data_, np.argmin(data_))
                data_ = np.delete(data_, np.argmax(data_))

        means.append(np.mean(data_))
        errors.append(np.std(data_) / len(data_)**0.5)
        
    return np.array(means).reshape(shape), np.array(errors).reshape(shape)
    
    
model_fid_mean, model_fid_std = mean_err_without_outliers(model_gen_fids)
model_reco_error_mean, model_reco_error_std = mean_err_without_outliers(model_mean_reco_errors)


## Best metrics

In [38]:
best_fid = np.nanargmin(model_fid_mean)
print(algo_labels[best_fid])

best_reco = np.nanargmin(np.where(model_mean_reco_error_mean > 1.e-3, model_mean_reco_error_mean, np.nan))
print(algo_labels[best_reco])


\af{}
\mfmfmd{sequential }
\mfmfescandal{alternating }


/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in greater
  


## Print result table

In [39]:
def print_results(
    l_label=max([len(l) for l in algo_labels]), l_mean=6, l_err=5, latex=False, after_dec=(1,1)
):
    # Number of digits
    l_result = l_mean + 2 + l_err
    l_total = l_label + 1 + 2 * (3 + l_result)
        
    # Divider
    empty_result = "" if latex else " "*(l_result + 1)
    col_divider = "&" if latex else "|"
    line_end = r"\\" if latex else ""
    block_divider = r"\midrule" if latex else "-"*l_total
    
    # Number formatting
    def _f(val, err, after_dec, best=False):
        if not np.any(np.isfinite(val)):
            return empty_result
        
        result = "{:>{}.{}f}".format(val, l_mean, after_dec)
        if latex and best:
            result = r"\textbf{" + result + "}"
            
        err_str = "({:>{}.{}f})".format(err, l_err, after_dec)
        if latex:
            result += r"\,\textcolor{darkgray}{" + err_str + "}"
        else:
            result += err_str
            
        result += "*" if not latex and best else " "
        
        if latex:
            result = result.replace("-", "$-{}$")
            result = result.replace("darkgray", "dark-gray")
        return result
    
    
    # Header
    print(f"{'':<{l_label}.{l_label}s} {col_divider} {"FID":>{l_result}.{l_result}s}{col_divider} {"Reco error":>{l_result}.{l_result}s} {line_end}")
    print(block_divider)

    # Iterate over methods
    for i, (label, fid, fid_err, reco, reco_err) in enumerate(zip(
        algo_labels, model_fid_mean, model_fid_std, model_reco_error_mean, model_reco_error_std
    )):
        # Divider
        if i in algo_dividers:
            print(block_divider)
            
        # Print results
        print(
            f"{label:<{l_label}.{l_label}s} {col_divider} "
            + f"{_f(fid, fid_err, after_decs[0], i==best_fid):>{l_result}.{l_result}s}{col_divider} "
            + f"{_f(reco, reco_err, after_decs[1], i==best_reco):>{l_result}.{l_result}s} {line_end}"
        )


In [40]:
print_results()

                                  |       Closure |    Reco error |         log p 
----------------------------------------------------------------------------------
\af{}                             |  0.014(0.002)*|  0.000(0.000) | -10.55( 3.42)  
\pie{} (original)                 |               |               |                
\pie{} (unconditional manifold)   |  0.036(0.006) |  4.553(0.238) |  -1.78( 0.14)  
\mfmfmd{sequential }              |  0.047(0.011) |  0.322(0.137)*|  -1.80( 0.09)  
\mfmfmd{alternating }             |  0.155(0.014) |  0.627(0.126) |  -2.30( 0.06)  
\mfmfemd{sequential }             |  0.056(0.010) |               |  -2.57( 0.42)  
\mfmfemd{alternating }            |  0.056(0.011) |               |  -2.42( 0.31)  
----------------------------------------------------------------------------------
\afscandal{}                      |  0.062(0.004) |  0.000(0.000) |   0.04( 0.11)  
\piescandal{, original}           |               |               |            

In [41]:
print_results(latex=True)

                                  &       Closure &    Reco error &         log p \\
\midrule
\af{}                             & \textbf{ 0.014}\,\textcolor{dark-gray}{(0.002)} &  0.000\,\textcolor{dark-gray}{(0.000)} & $-{}$10.55\,\textcolor{dark-gray}{( 3.42)}  \\
\pie{} (original)                 & & &  \\
\pie{} (unconditional manifold)   &  0.036\,\textcolor{dark-gray}{(0.006)} &  4.553\,\textcolor{dark-gray}{(0.238)} &  $-{}$1.78\,\textcolor{dark-gray}{( 0.14)}  \\
\mfmfmd{sequential }              &  0.047\,\textcolor{dark-gray}{(0.011)} & \textbf{ 0.322}\,\textcolor{dark-gray}{(0.137)} &  $-{}$1.80\,\textcolor{dark-gray}{( 0.09)}  \\
\mfmfmd{alternating }             &  0.155\,\textcolor{dark-gray}{(0.014)} &  0.627\,\textcolor{dark-gray}{(0.126)} &  $-{}$2.30\,\textcolor{dark-gray}{( 0.06)}  \\
\mfmfemd{sequential }             &  0.056\,\textcolor{dark-gray}{(0.010)} & &  $-{}$2.57\,\textcolor{dark-gray}{( 0.42)}  \\
\mfmfemd{alternating }            &  0.056\,\textcolor{dar